# **Importing the Dataset from Kaggle**

In this section, we connect Google Drive to access the Kaggle API token stored inside it.  
Using the Kaggle API, we import the **Superstore Sales Dataset**, which is a retail dataset of a global superstore collected over four years.  
After downloading, we unzip the dataset files so that we can start exploring and analyzing them with PySpark.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d rohitsahoo/sales-forecasting

Dataset URL: https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting
License(s): GPL-2.0
  0% 0.00/480k [00:00<?, ?B/s]
100% 480k/480k [00:00<00:00, 966MB/s]


In [5]:
!unzip -o sales-forecasting.zip

Archive:  sales-forecasting.zip
  inflating: train.csv               


# **Reading CSV File**

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()

In [7]:
# Define the dataset path
path = "/content/train.csv"

# Read the CSV file into a Spark DataFrame
df = spark.read.csv(path, header=True, inferSchema=True)

# Display the first 5 rows of the DataFrame
df.show(5)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+
|     1|CA-2017-152156|08/11/2017|11/11/2017|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset Col...|  261.96|
|     2|CA-2017-152156|08/11/2017|11/11/2017|  Second Class|   C

In [8]:
# Number of rows
df.count()

9800

In [9]:
# To see basic statistics of numeric columns
df.describe().show()

+-------+------------------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+---------------+----------+------------+--------------------+-----------------+
|summary|            Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|    City|  State|       Postal Code| Region|     Product ID|  Category|Sub-Category|        Product Name|            Sales|
+-------+------------------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+---------------+----------+------------+--------------------+-----------------+
|  count|              9800|          9800|      9800|      9800|          9800|       9800|              9800|       9800|         9800|    9800|   9800|              9789|   9800|           9800|      9800|        9

In [10]:
# Print the schema (structure) of the DataFrame
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)



In [11]:
# Check column names with their data types
df.dtypes

[('Row ID', 'int'),
 ('Order ID', 'string'),
 ('Order Date', 'string'),
 ('Ship Date', 'string'),
 ('Ship Mode', 'string'),
 ('Customer ID', 'string'),
 ('Customer Name', 'string'),
 ('Segment', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Postal Code', 'int'),
 ('Region', 'string'),
 ('Product ID', 'string'),
 ('Category', 'string'),
 ('Sub-Category', 'string'),
 ('Product Name', 'string'),
 ('Sales', 'string')]

The **Segment** column has 3 unique values: *Consumer*, *Corporate*, and *Home Office*.

- **Consumer**: individual buyers purchasing products for personal use.  
- **Corporate**: businesses or companies purchasing in bulk for their offices or employees.  
- **Home Office**: small or self-employed individuals running a business from home, such as freelancers or small startups.

In [12]:
# Get unique values from the Segment column
df.select("Segment").distinct().show()

+-----------+
|    Segment|
+-----------+
|   Consumer|
|Home Office|
|  Corporate|
+-----------+



In this section, we correct the data types of some columns.  
As you can see, the columns **"Order Date"** and **"Ship Date"** were strings but should be converted to dates.  
The column **"Postal Code"** should be an integer, and **"Sales"** should be a double for numerical calculations.

In [13]:
# Change the types of the columns to their correct types
from pyspark.sql.functions import col, to_date

# Convert numeric columns
df = df.withColumn("Sales", col("Sales").cast("double"))
df = df.withColumn("Postal Code", col("Postal Code").cast("int"))

# Convert date columns (notice the corrected quotes and format)
df = df.withColumn("Order Date", to_date(col("Order Date"), "dd/MM/yyyy"))
df = df.withColumn("Ship Date", to_date(col("Ship Date"), "dd/MM/yyyy"))

In [14]:
# Check column names with their data types
df.dtypes

[('Row ID', 'int'),
 ('Order ID', 'string'),
 ('Order Date', 'date'),
 ('Ship Date', 'date'),
 ('Ship Mode', 'string'),
 ('Customer ID', 'string'),
 ('Customer Name', 'string'),
 ('Segment', 'string'),
 ('Country', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Postal Code', 'int'),
 ('Region', 'string'),
 ('Product ID', 'string'),
 ('Category', 'string'),
 ('Sub-Category', 'string'),
 ('Product Name', 'string'),
 ('Sales', 'double')]

In [15]:
# To check for missing or null values in selected columns
def check_nulls(df):
    for c in df.columns:
        num_null = df.filter((col(c).isNull()) | (col(c) == "")).count()
        print(f"The column '{c}' has {num_null} null values.")

check_nulls(df)

The column 'Row ID' has 0 null values.
The column 'Order ID' has 0 null values.
The column 'Order Date' has 0 null values.
The column 'Ship Date' has 0 null values.
The column 'Ship Mode' has 0 null values.
The column 'Customer ID' has 0 null values.
The column 'Customer Name' has 0 null values.
The column 'Segment' has 0 null values.
The column 'Country' has 0 null values.
The column 'City' has 0 null values.
The column 'State' has 0 null values.
The column 'Postal Code' has 11 null values.
The column 'Region' has 0 null values.
The column 'Product ID' has 0 null values.
The column 'Category' has 0 null values.
The column 'Sub-Category' has 0 null values.
The column 'Product Name' has 0 null values.
The column 'Sales' has 292 null values.


*   The **Sales** column is very important for our analysis, so we will drop any rows that contain null values in this column.
*   On the other hand, the **Postal Code** column is not as critical since we already have information about the city, state, and region. Therefore, we will replace its null values with a placeholder value of 0.


In [16]:
# Drop the rows where 'Sales' has null values
df = df.na.drop(subset=['Sales'])

# Replace null values in 'Postal Code' with a placeholder value of 0
df = df.na.fill({'Postal Code': 0})

In [17]:
check_nulls(df)

The column 'Row ID' has 0 null values.
The column 'Order ID' has 0 null values.
The column 'Order Date' has 0 null values.
The column 'Ship Date' has 0 null values.
The column 'Ship Mode' has 0 null values.
The column 'Customer ID' has 0 null values.
The column 'Customer Name' has 0 null values.
The column 'Segment' has 0 null values.
The column 'Country' has 0 null values.
The column 'City' has 0 null values.
The column 'State' has 0 null values.
The column 'Postal Code' has 0 null values.
The column 'Region' has 0 null values.
The column 'Product ID' has 0 null values.
The column 'Category' has 0 null values.
The column 'Sub-Category' has 0 null values.
The column 'Product Name' has 0 null values.
The column 'Sales' has 0 null values.


# **What are the total sales by product category?**

We want to know which product categories bring the most revenue.

In [18]:
from pyspark.sql import functions as F

df.groupBy("Category") \
  .agg(F.sum("Sales").alias("Total_Sales")) \
  .orderBy(F.desc("Total_Sales")) \
  .show()

+---------------+-----------------+
|       Category|      Total_Sales|
+---------------+-----------------+
|     Technology|827201.9069999964|
|      Furniture|719791.4556999996|
|Office Supplies|690139.8000000035|
+---------------+-----------------+



# **What are the average sales by region?**

In [19]:
df.groupBy("Region") \
  .agg(F.avg("Sales").alias("Avg_Sales")) \
  .orderBy(F.desc("Avg_Sales")) \
  .show()

+-------+------------------+
| Region|         Avg_Sales|
+-------+------------------+
|  South|248.17799550417496|
|   East|245.93614910979176|
|   West| 229.8731976629362|
|Central|220.71330230040624|
+-------+------------------+



# **Which cities have the highest number of orders?**

In [20]:
df.groupBy("City") \
  .agg(F.count("Order ID").alias("Total_Orders")) \
  .orderBy(F.desc("Total_Orders")) \
  .show()

+-------------+------------+
|         City|Total_Orders|
+-------------+------------+
|New York City|         869|
|  Los Angeles|         712|
| Philadelphia|         503|
|San Francisco|         485|
|      Seattle|         410|
|      Houston|         367|
|      Chicago|         295|
|     Columbus|         213|
|    San Diego|         166|
|  Springfield|         156|
|       Dallas|         148|
| Jacksonville|         121|
|      Detroit|         111|
|       Newark|          90|
|     Richmond|          79|
|      Jackson|          79|
|     Columbia|          78|
|       Aurora|          67|
|      Phoenix|          61|
|    Arlington|          59|
+-------------+------------+
only showing top 20 rows



# **Which shipping mode generates the highest total sales?**

In [21]:
df.groupBy("Ship Mode") \
  .agg(F.sum("Sales").alias("Total_Sales")) \
  .orderBy(F.desc("Total_Sales")) \
  .show()

+--------------+------------------+
|     Ship Mode|       Total_Sales|
+--------------+------------------+
|Standard Class|1325110.5679999827|
|  Second Class|444062.46040000016|
|   First Class|343716.61730000016|
|      Same Day|        124243.517|
+--------------+------------------+



# **Which customer generated the highest total sales?**

In [22]:
df.groupBy("Customer ID", "Customer Name") \
  .agg(F.sum("Sales").alias("Total_Sales")) \
  .orderBy(F.desc("Total_Sales")) \
  .show()

+-----------+------------------+------------------+
|Customer ID|     Customer Name|       Total_Sales|
+-----------+------------------+------------------+
|   SM-20320|       Sean Miller|          25043.05|
|   TC-20980|      Tamara Chand|19017.847999999998|
|   RB-19360|      Raymond Buch|         15117.339|
|   TA-21385|      Tom Ashbrook|          14595.62|
|   AB-10105|     Adrian Barton|14355.610999999997|
|   SC-20095|      Sanjit Chand|14142.333999999999|
|   KL-16645|      Ken Lonsdale|         14071.917|
|   HL-15040|      Hunter Lopez|12873.297999999999|
|   SE-20110|      Sanjit Engle|12209.438000000002|
|   CC-12370|Christopher Conant|         12129.072|
|   TS-21370|      Todd Sumrall|         11885.871|
|   GT-14710|         Greg Tran|11820.119999999997|
|   BM-11140|      Becky Martin|11609.900000000001|
|   SV-20365|       Seth Vernon|11420.645999999997|
|   CJ-12010|   Caroline Jumper|         11079.742|
|   CL-12565|       Clay Ludtke|10880.545999999998|
|   ME-17320

# **How do sales vary over time?**

In [23]:
df.groupBy(
    F.year(F.col("Order Date")).alias("Year"),
    F.month(F.col("Order Date")).alias("Month")
).agg(
    F.sum("Sales").alias("Total_Sales")
).orderBy(F.desc("Year"), F.desc("Month")).show()

+----+-----+------------------+
|Year|Month|       Total_Sales|
+----+-----+------------------+
|2018|   12|        80428.6858|
|2018|   11|116873.71100000002|
|2018|   10| 77213.69119999997|
|2018|    9| 84870.84200000003|
|2018|    8| 62360.36799999999|
|2018|    7|44051.136000000006|
|2018|    6|47451.563700000006|
|2018|    5|        43660.2242|
|2018|    4| 35040.54710000001|
|2018|    3| 58730.28379999998|
|2018|    2|        19882.1864|
|2018|    1|         43365.386|
|2017|   12| 94815.14500000002|
|2017|   11|        78109.4318|
|2017|   10| 58032.10300000001|
|2017|    9| 67691.44689999995|
|2017|    8|30441.228300000002|
|2017|    7| 38166.82799999997|
|2017|    6| 39008.14399999998|
|2017|    5|56126.499999999985|
+----+-----+------------------+
only showing top 20 rows



# **Which product sub-category has the highest average sales per order?**

In [24]:
df.groupBy("Sub-Category") \
  .agg(F.avg("Sales").alias("Avg_Sales")) \
  .orderBy(F.desc("Avg_Sales")) \
  .show()

+------------+------------------+
|Sub-Category|         Avg_Sales|
+------------+------------------+
|     Copiers|2215.8802121212116|
|    Machines| 1645.553313043478|
|      Tables| 645.8937197452233|
|      Chairs| 531.8331647446471|
|   Bookcases|503.59822433628307|
|      Phones|377.33696082949336|
|    Supplies| 266.5822093023256|
|     Storage| 263.0768228004958|
|  Appliances|227.92680392156856|
| Accessories|217.17817460317482|
|     Binders|140.08422024233784|
| Furnishings| 90.68272911963885|
|   Envelopes|  62.4630583333333|
|       Paper| 59.61025363489499|
|      Labels|34.587467787114846|
|         Art| 34.01963057324835|
|   Fasteners|14.288688995215308|
+------------+------------------+



# **Which region has the highest total number of orders?**

In [25]:
df.groupBy("Region") \
  .agg(F.count("Order ID").alias("Total_Orders")) \
  .orderBy(F.desc("Total_Orders")) \
  .show()

+-------+------------+
| Region|Total_Orders|
+-------+------------+
|   West|        3038|
|   East|        2696|
|Central|        2217|
|  South|        1557|
+-------+------------+



# **Find the top 3 products with the highest average sales in each Category?**

In [26]:
from pyspark.sql.window import Window

df_avg = df.groupBy("Category", "Product Name").agg(F.avg("Sales").alias("Avg_Sales"))


windowSpec = Window.partitionBy("Category").orderBy(F.desc("Avg_Sales"))

df_ranked = df_avg.withColumn("Rank", F.row_number().over(windowSpec))

df_top3 = df_ranked.filter(F.col("Rank") <= 3)
df_top3.show()

+---------------+--------------------+-----------------+----+
|       Category|        Product Name|        Avg_Sales|Rank|
+---------------+--------------------+-----------------+----+
|      Furniture|Riverside Palais ...|       3122.19312|   1|
|      Furniture|HON 5400 Series T...|         2733.822|   2|
|      Furniture|Chromcraft Bull-N...|         2396.763|   3|
|Office Supplies|High Speed Automa...|5676.770666666666|   1|
|Office Supplies|Ibico EPK-21 Elec...|5291.972000000001|   2|
|Office Supplies|GBC DocuBind P400...|         2994.178|   3|
|     Technology|Cisco TelePresenc...|         22638.48|   1|
|     Technology|Canon imageCLASS ...|       12319.9648|   2|
|     Technology|Cubify CubeX 3D P...|          7999.98|   3|
+---------------+--------------------+-----------------+----+



# **For each region, find the most recent order placed and the details of that order**

In [27]:
windowSpec = Window.partitionBy('Region').orderBy(F.desc('Order Date'))

df_recentOrder = df.withColumn('Rank of Orders', F.row_number().over(windowSpec))
df_recentOrder.filter(F.col("Rank of Orders") == 1).show()

+------+--------------+----------+----------+--------------+-----------+-----------------+-----------+-------------+-------------+----------+-----------+-------+---------------+---------------+------------+--------------------+-------+--------------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|    Customer Name|    Segment|      Country|         City|     State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|  Sales|Rank of Orders|
+------+--------------+----------+----------+--------------+-----------+-----------------+-----------+-------------+-------------+----------+-----------+-------+---------------+---------------+------------+--------------------+-------+--------------+
|   646|CA-2018-126221|2018-12-30|2019-01-05|Standard Class|   CC-12430|      Chuck Clark|Home Office|United States|     Columbus|   Indiana|      47201|Central|OFF-AP-10002457|Office Supplies|  Appliances|Eureka The Boss P...|  209.3|            

In [28]:
df_recentOrder.select("Region", "Order Date", "Order ID", "Customer Name", "Rank of Orders").filter(F.col("Rank of Orders") == 1).show()

+-------+----------+--------------+-----------------+--------------+
| Region|Order Date|      Order ID|    Customer Name|Rank of Orders|
+-------+----------+--------------+-----------------+--------------+
|Central|2018-12-30|CA-2018-126221|      Chuck Clark|             1|
|   East|2018-12-30|CA-2018-143259|Patrick O'Donnell|             1|
|  South|2018-12-29|US-2018-158526| Katherine Hughes|             1|
|   West|2018-12-30|CA-2018-115427|       Erica Bern|             1|
+-------+----------+--------------+-----------------+--------------+



# **For each customer, find their most expensive order based on the Sales value?**

In [31]:
windowSpec1 = Window.partitionBy("Customer ID").orderBy(F.desc("Sales"))
df1 = df.withColumn("Rank of Expensive Orders", F.row_number().over(windowSpec1))
df1.select('Customer Name', 'Order ID', 'Product Name', 'Sales').filter(F.col("Rank of Expensive Orders") == 1).show()

+--------------------+--------------+--------------------+--------+
|       Customer Name|      Order ID|        Product Name|   Sales|
+--------------------+--------------+--------------------+--------+
|          Alex Avila|CA-2017-103982|High Speed Automa...|3930.072|
|        Allen Armold|CA-2017-131065|Logitech diNovo E...|  499.98|
|        Andrew Allen|CA-2017-114601|NETGEAR AC1750 Du...|  479.97|
|       Anna Andreadi|CA-2016-110863|Hon Olson Stacker...|  1323.9|
|       Aaron Bergman|CA-2017-140935|Sauder Facets Col...|  341.96|
|     Adam Bellavance|CA-2017-129714|GBC DocuBind P400...|4355.168|
|       Adrian Barton|CA-2017-117121|GBC Ibimaster 500...| 9892.74|
|         Aimee Bixby|CA-2015-169061|Logitech Wireless...|  479.97|
|         Alan Barnes|CA-2016-115945|Tennsco Regal She...|  304.23|
|Alejandro Ballentine|CA-2018-126662|Brother DCP1000 D...| 479.984|
|           Ann Blume|US-2018-155425|Hon 2090 “Pillow ...| 899.136|
|       Alyssa Crouse|CA-2015-129924|Hon 5100 Se

# **For each region, which product category has the highest total sales contribution?**

In [40]:
df2 = df.groupBy('Region', 'Category') \
        .agg(F.sum('Sales').alias('Total Sales Per Category'))

windowSpec2 = Window.partitionBy('Region').orderBy(F.desc('Total Sales Per Category'))

df_highest_sales = df2.withColumn('Rank Of Product Category In Each Region', F.row_number().over(windowSpec2))

df_highest_sales.filter(F.col('Rank Of Product Category In Each Region') == 1) \
                .select('Region', 'Category', 'Total Sales Per Category') \
                .show()

+-------+----------+------------------------+
| Region|  Category|Total Sales Per Category|
+-------+----------+------------------------+
|Central|Technology|      168724.42799999993|
|   East|Technology|      263014.62900000036|
|  South|Technology|      148153.82399999994|
|   West|Technology|      247309.02599999995|
+-------+----------+------------------------+

